# Steam

__author__ = "Andres Ardila, Johan Triviño, Jenny Gamboa"
__maintainer__ = "Proyecto de Profundización"
__copyright__ = "Copyright 2023"
__version__ = "0.0.1"

Técnica del procesamiento de lenguaje natural, usado principalmente para la preparación de texto o documentos, para su posterior procesamiento, su principal función es reducir las palabras a su forma raiz y asignar las palabras a esta forma.  

- Librerias requeridas para el modelo.

## TF-IDF

Modelo encargado identificar las palabras mas relevantes de un texto para así asociar este documento a un tema en especifico o simplemente saber con cuales palabras tiene mas relación.

In [ ]:
import pandas as pd
import re 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=set(stopwords.words('spanish'))
from nltk.stem import SnowballStemmer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from ipywidgets import interact
import ipywidgets as widgets
from IPython.core.display import display,HTML
from google.colab import files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


- Comunicación con drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- LLamado del archivo csv del normograma.

In [ ]:
url ="/content/drive/MyDrive/concatenado/procesado_normas.json"

In [ ]:
Norma=pd.read_json(url, encoding='utf-8-sig')
Norma.shape

(665, 4)

In [ ]:
Norma

,index,No_Articulo,Contenido_Articulo,Norma
0,0,Artículo 1.,Artículo 1. Publicidad de proyectos de pliegos...,Decreto_2170_de_2002
1,1,Artículo 11.,Artículo 11. Menor Cuantía. Para la celebració...,Decreto_2170_de_2002
2,2,Artículo 12.,Artículo 12. Del sistema de conformación dinám...,Decreto_2170_de_2002
3,3,Artículo 13.,Artículo 13. De los contratos de prestación de...,Decreto_2170_de_2002
4,4,Artículo 14.,"Artículo 14. Modificado por el art. 1, Decreto...",Decreto_2170_de_2002
...,...,...,...,...
660,84,ARTÍCULO 81º.,ARTÍCULO 81º. De la Derogatoria y de la Vigenc...,Ley_80_de_1993
661,85,ARTÍCULO 8º.,ARTÍCULO 8º. De las Inhabilidades e Incompatib...,Ley_80_de_1993
662,86,ARTÍCULO 9A.,"ARTÍCULO 9A. Adicionado por el art. 8, Ley 201...",Ley_80_de_1993
663,87,ARTÍCULO 9º.,ARTÍCULO 9º. De las Inhabilidades e Incompatib...,Ley_80_de_1993


- Limpieza de columnas que no influyen en el análisis 

In [ ]:
Norma.drop(['index'], axis = 'columns', inplace=True)
Norma

,No_Articulo,Contenido_Articulo,Norma
0,Artículo 1.,Artículo 1. Publicidad de proyectos de pliegos...,Decreto_2170_de_2002
1,Artículo 11.,Artículo 11. Menor Cuantía. Para la celebració...,Decreto_2170_de_2002
2,Artículo 12.,Artículo 12. Del sistema de conformación dinám...,Decreto_2170_de_2002
3,Artículo 13.,Artículo 13. De los contratos de prestación de...,Decreto_2170_de_2002
4,Artículo 14.,"Artículo 14. Modificado por el art. 1, Decreto...",Decreto_2170_de_2002
...,...,...,...
660,ARTÍCULO 81º.,ARTÍCULO 81º. De la Derogatoria y de la Vigenc...,Ley_80_de_1993
661,ARTÍCULO 8º.,ARTÍCULO 8º. De las Inhabilidades e Incompatib...,Ley_80_de_1993
662,ARTÍCULO 9A.,"ARTÍCULO 9A. Adicionado por el art. 8, Ley 201...",Ley_80_de_1993
663,ARTÍCULO 9º.,ARTÍCULO 9º. De las Inhabilidades e Incompatib...,Ley_80_de_1993


- Copia del dataframe original. 

In [ ]:
Norma1=Norma.copy()

- Función para la aplicación del steaming al corpus del texto. 

In [ ]:
def steamer(Corpus):
        
  Corpus=[i.lower() for i in Corpus]
  stem=SnowballStemmer('spanish')
  
  #Corpus=[' '.join([j for j in i.split() if  j not in stopwords])  for i in Corpus]
  
  Corpus=[' '.join([stem.stem(j) for j in i.split()]) for i in Corpus]
  return Corpus

- Ciclo con la funcion de guardar las normas en minúscula, en una lista. 

In [ ]:
Norma2 = []
for j,i in enumerate(Norma1["Contenido_Articulo"]):
    if type(i)==str:
        Norma2.append(i.lower()) 
    #print(j)

- Ciclo para agregar una columna al dataframe con el steaming aplicado. 

In [ ]:
for i in range (len(Norma1['Contenido_Articulo'])):
  Norma1['Contenido_Articulo'][i] =pd.Series(Norma1['Contenido_Articulo'][i])
  Corpus= steamer(Norma1['Contenido_Articulo'][i])
  Norma1['Contenido_Articulo'][i]= Corpus[0]


In [ ]:
print(len(Norma1))

665


In [ ]:
len(Norma2)

- Ciclo para formar una lista de la norma sin stopwords y en minúscula. 

In [ ]:
Normasinparada=[]
for i,noti in enumerate(Norma2):
    Normasinparada.append([j.lower() for j in noti.split() if not j.lower() in stopwords and len(j)>1])

In [ ]:
Normasinparada2=[' '.join(i) for i in Normasinparada]
len(Normasinparada2)

- Aplicación de vectorización por el metodo Tfidf

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(Normasinparada2)
X

In [ ]:
y = vectorizer.transform(Normasinparada2)
y

* Distancia Euclidiana

In [ ]:
distancias_y = euclidean_distances(X,y)
len(distancias_y)

- Agregar columna con el contenido sin steam. 

In [ ]:
Norma1['Contenido_Articulo_Completo']=Norma['Contenido_Articulo']

- Función para la busqueda de los artículos relacionados, partiendo frases clave. 

In [ ]:

def funcion_recomendacion(entrada, Number):
  z=int(Number)
  y=vectorizer.transform([entrada])
  distancias_y=cosine_distances(X,y)  
  ind=np.argsort(distancias_y,axis=0).reshape(Norma.shape[0])[:z]
  return Norma1[['Norma','No_Articulo','Contenido_Articulo_Completo']].loc[ind]


- Ejecución del modelo de búsqueda. 

In [ ]:
interact(funcion_recomendacion,entrada="", Number=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])

In [ ]:
#z=int(Number)
y=vectorizer.transform(['¿cuándo es un contrato de mínima cuantía?'])
distancias_y=cosine_distances(X,y)  
ind=np.argsort(distancias_y,axis=0).reshape(Norma.shape[0])[:20]
Nor = Norma1[['Norma','No_Articulo','Contenido_Articulo_Completo']].loc[ind]

In [ ]:
Nor

In [ ]:
Nor.to_csv('SteamTfidf.csv', encoding='utf-8-sig')
files.download('SteamTfidf.csv')

In [ ]:
text= "".join(Contenido_Articulo_Completo for Contenido_Articulo_Completo in Nor.Contenido_Articulo_Completo)

In [ ]:
wordcloud = WordCloud(width=800, height=400, max_font_size=100, background_color="white").generate(text)

In [ ]:
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()